# Creating Dataset

To creat a dataset, I scrape reviews from [seek.com.au](https://www.seek.com.au/) for two companies __`Woolworths`__ and __`Coles`__ which both have a fair number of reviews.

To avoid `JavaScript` obstructions (downloading webpage source before target content is loaded) I use __`selenium`__ along with __`BeautifulSoup`__.

In [2]:
# imports

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from datetime import datetime

import csv

### Woolworths Reviews

Having a look at Woolworth [reviews](https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews) it can be observed that:
1. reviews are delivered over __96__ different pages
2. each review link has `html tag <a>` and `class = '_2gLKVsp'`

In [3]:
content = requests.get('https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews')
soup = BeautifulSoup(content.text, 'html.parser')
links = soup.find('div', {'class' : '_1Y5kUMd'}).find_all('a', {'class' : '_2gLKVsp'})
links[0]['href']

'/companies/woolworths-supermarkets-432295/reviews/2276877'

Now let's scrape all the review links

In [4]:
browser = webdriver.Firefox()
woolies = []

for page in range(1, 97):
    url = 'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews?page={}'.format(page)
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find('div', {'class' : '_1Y5kUMd'}).find_all('a', {'class' : '_2gLKVsp'})
        
    for link in links:
        woolies.append("https://www.seek.com.au{}".format(link['href']))

To save all the 1908 links in a file:

In [5]:
with open('woolies_urls.txt', 'w') as file:
    for url in woolies:
        file.write('{}\n'.format(url))

In [102]:
woolies[0:5]

['https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/2276877',
 'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/256376',
 'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/340048',
 'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/277368',
 'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/197028']

During scraping below, it happens often that process becomes interrupted probably because site is lagging behind to load completely while data is fetched. However, with one of errors it became known that one of linkes is faulty and should be removed from the list:

`woolies[1277]`  
`https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/35519`

In [106]:
woolies.pop(1277)

'https://www.seek.com.au/companies/woolworths-supermarkets-432295/reviews/35519'

Let's scrape reviews and some other data and save them in a `csv` format file.

In [ ]:
browser = webdriver.Firefox()

with open('RawData.csv', 'a+', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for url in woolies:
        browser.get(url)
        review = browser.page_source
        soup = BeautifulSoup(review, 'html.parser')

        company = 'Woolworths'
        review = soup.find('span', {'data-automation' : 'reviewCardFull-pros'}).text + "\n" + soup.find(
            'span', {'data-automation' : 'reviewCardFull-cons'}).text
        rating = soup.find('span', {'class' : '_1erK2ob'}).text
        review_date = soup.find('span', {'class' : '_3FrNV7v _38Keb0I _2QG7TNq E6m4BZb'}).text
        time_stamp = datetime.today().strftime('%Y, %B')
        
        datawriter.writerow([company, review, rating, review_date, time_stamp])

### Coles Reviews

Time to run same process for `Coles`. So:
1. `tags` and `classes` are the same.
2. reviews are delivered upon 104 pages

In [7]:
browser = webdriver.Firefox()
coles = []

for page in range(1, 105):
    url = 'https://www.seek.com.au/companies/coles-supermarkets-432309/reviews?page={}'.format(page)
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find('div', {'class' : '_1Y5kUMd'}).find_all('a', {'class' : '_2gLKVsp'})
        
    for link in links:
        coles.append("https://www.seek.com.au{}".format(link['href']))

Let's save all the review's link in a text file:

In [9]:
with open('coles_urls.txt', 'w+') as file:
    for url in coles:
        file.write('{}\n'.format(url))

Then, time to collect reviews and add it to the raw dataset:

following links are fulty and should be removed:  
coles[245]
https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/61321  
coles[794]
https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/40365  
coles[1600]
https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/52050  
coles[1704]
https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/40431  
coles[1712]
https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/52054

In [ ]:
coles.remove('https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/61321')
coles.remove('https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/40365')
coles.remove('https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/52050')
coles.remove('https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/40431')
coles.remove('https://www.seek.com.au/companies/coles-supermarkets-432309/reviews/52054')

In [ ]:
browser = webdriver.Firefox()

with open('RawData.csv', 'a+', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for url in coles:
        browser.get(url)
        review = browser.page_source
        soup = BeautifulSoup(review, 'html.parser')

        company = 'Coles'
        review = soup.find('span', {'data-automation' : 'reviewCardFull-pros'}).text + "\n" + soup.find(
            'span', {'data-automation' : 'reviewCardFull-cons'}).text
        rating = soup.find('span', {'class' : '_1erK2ob'}).text
        review_date = soup.find('span', {'class' : '_3FrNV7v _38Keb0I _2QG7TNq E6m4BZb'}).text
        time_stamp = datetime.today().strftime('%Y, %B')
        
        datawriter.writerow([company, review, rating, review_date, time_stamp])

### ALDI Reviews

In [4]:
# fetching all urls for aldi's reviews

browser = webdriver.Firefox()
ALDI = []

for page in range(25, 26):
    url = 'https://www.seek.com.au/companies/aldi-432489/reviews?page={}'.format(page)
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find('div', {'class' : '_1Y5kUMd'}).find_all('a', {'class' : '_2gLKVsp'})
        
    for link in links:
        ALDI.append("https://www.seek.com.au{}".format(link['href']))

In [6]:
# writing aldi's urls into a text file

with open('aldi.txt', 'w+') as file:
    for url in ALDI:
        file.write('{}\n'.format(url))

In [18]:
# faulty links to remove
ALDI.remove('https://www.seek.com.au/companies/aldi-432489/reviews/61903')
ALDI.remove('https://www.seek.com.au/companies/aldi-432489/reviews/40163')

In [ ]:
browser = webdriver.Firefox()

with open('RawData.csv', 'a+', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for url in ALDI[344:]:
        browser.get(url)
        review = browser.page_source
        soup = BeautifulSoup(review, 'html.parser')
        
        company = 'ALDI'
        review = soup.find('span', {'data-automation' : 'reviewCardFull-pros'}).text + "\n" + soup.find(
            'span', {'data-automation' : 'reviewCardFull-cons'}).text
        rating = soup.find('span', {'class' : '_1erK2ob'}).text
        review_date = soup.find('span', {'class' : '_3FrNV7v _38Keb0I _2QG7TNq E6m4BZb'}).text
        time_stamp = datetime.today().strftime('%Y, %B')
        
        datawriter.writerow([company, review, rating, review_date, time_stamp])

Let's see what has been collected so far: `(4,468 reviews)`


## Kmart

In [8]:
# fetching all urls for Kmart's reviews

browser = webdriver.Firefox()
kmart = []

for page in range(1, 40):
    url = 'https://www.seek.com.au/companies/kmart-432302/reviews?page={}'.format(page)
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find('div', {'class' : '_1Y5kUMd'}).find_all('a', {'class' : '_2gLKVsp'})
        
    for link in links:
        kmart.append("https://www.seek.com.au{}".format(link['href']))

In [11]:
# writing kmarts's urls into a text file

with open('kmart.txt', 'w+') as file:
    for url in kmart:
        file.write('{}\n'.format(url))

In [20]:
# faulty links to remove
kmart.remove('https://www.seek.com.au/companies/kmart-432302/reviews/41755')

In [ ]:
browser = webdriver.Firefox()

counter = 574

with open('RawData.csv', 'a+', newline='') as csvfile:
    datawriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for url in kmart[574:]:
        browser.get(url)
        review = browser.page_source
        soup = BeautifulSoup(review, 'html.parser')
        
        print(counter, end='\t')
        counter += 1
        
        company = 'Kmart'
        review = soup.find('span', {'data-automation' : 'reviewCardFull-pros'}).text + "\n" + soup.find(
            'span', {'data-automation' : 'reviewCardFull-cons'}).text
        rating = soup.find('span', {'class' : '_1erK2ob'}).text
        review_date = soup.find('span', {'class' : '_3FrNV7v _38Keb0I _2QG7TNq E6m4BZb'}).text
        time_stamp = datetime.today().strftime('%Y, %B')
        
        datawriter.writerow([company, review, rating, review_date, time_stamp])

In [4]:
import pandas as pd

In [21]:
df = pd.read_csv('RawData.csv', header=None)

In [22]:
df.head()

,0,1,2,3,4
0,Woolworths,-Great working environment with very good supp...,5.0,2 years ago,"2020, January"
1,Woolworths,"I enjoyed what I am doing, it's a tough job, b...",3.0,4 years ago,"2020, January"
2,Woolworths,Working with staff everyday. The ability to wo...,4.0,4 years ago,"2020, January"
3,Woolworths,Great opportunities for career advancement for...,4.0,4 years ago,"2020, January"
4,Woolworths,During peek sales periods; casuals get great h...,3.0,4 years ago,"2020, January"


In [23]:
df.shape

(5247, 5)